In [1]:
import re
import time
import math
import string
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.stem.rslp import RSLPStemmer
from nltk.stem import PorterStemmer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
import xgboost as xgb
from copy import deepcopy
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [2]:
#Carrega o dataset a partir de uma url da aws
df_original = pd.read_csv('https://s3.amazonaws.com/aulas-fiap/imdb-reviews-pt-br.csv')

#Exibe as características do dataset
df_original.describe()

,id
count,49459.000000
mean,24730.960917
std,14277.792868
min,1.000000
25%,12366.500000
50%,24731.000000
75%,37095.500000
max,49460.000000


In [3]:
df_original.head(5)

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
#Atribui o dataset que será trabalhado 
df = df_original 

#converte todas as palavras para minúsculo porque considero que não há diferença entre maiúsculas e minúsculas 
#para capturar o sentimento contido nas frases
df.text_pt = df.text_pt.str.lower()

In [5]:
#Vetoriza o texto utilizando TF-IDF em unigramas 
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df.text_pt)
text_vect = vect.transform(df.text_pt)

In [6]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [7]:
#Testa com Árvore de Decisão

tree = DecisionTreeClassifier(random_state=42) 

start = time.time()

tree.fit(X_train, y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = tree.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 80.0670280456543s
F1 Score: 0.7077


Gera um indicador F1 levemente acima do mínimo pedido pelo professor. Mínimo pedido = 70%. Obtido = <b>70,77 %</b>

In [8]:
#Testa com KNN

neigh = KNeighborsClassifier(n_neighbors=5)

start = time.time()

neigh.fit(X_train, y_train)
end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = neigh.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.07200050354003906s
F1 Score: 0.78


Obtido agora um F1 Score de <b>78%</b>. Melhor que árvore de decisão

In [9]:
#Testa com SVM SVC

# ATENÇÃO: Elevado tempo de execução 

svm_clf = SVC(C=100, kernel='linear',random_state =42)

start = time.time()

svm_clf.fit(X_train, y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_clf.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 5918.37014746666s
F1 Score: 0.8737


Ainda melhor, o modelo SVM SVC obteve um F1 Score de <b>87,37%</b>.

In [10]:
#Testa com SVM Linear

svm_linear = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear.fit(X_train, y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 8.073994636535645s
F1 Score: 0.8889


O melhor até então, o SVM Linear conseguiu <b>88,89%</b> de F1 Score. 

In [11]:
#Testa com Random forest

rand_forest = RandomForestClassifier(n_estimators=300,random_state=42,max_depth=30)

start = time.time()

rand_forest.fit(X_train, y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = rand_forest.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 102.53799772262573s
F1 Score: 0.8441


O Random Forest não teve um desempenho tão bom, ficou abaixo dos algoritmos de SVM. F1 Score de <b>84,41%</b>

In [12]:
#### Teste com Naive Bayes - Bernoulli ####

naive_berno = BernoulliNB()

start = time.time()

naive_berno.fit(X_train,y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_berno.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.15800118446350098s
F1 Score: 0.857


O algoritmo da Linha de Naive Bayes (probabilístico) é bem simples e ainda assim conseguiu um desempenho muito bom. 
F1 Score obtido é de <b>85,70%</b>

In [13]:
## Teste com outro algoritmo probabilístico da família Naive Bayes - Multinomial ###

naive_multi = MultinomialNB()

start = time.time()

naive_multi.fit(X_train,y_train)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.10699868202209473s
F1 Score: 0.8617


Assim como o outro algoritmo da mesma família, apresenta um bom desempenho e obtem um F1 score ligeiramente superior <b>86,17%</b>

In [14]:
#Teste com o XGBBoost - XGBClassifier #

def xgb_f1(y,t):
    t = t.get_label()
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in y] # arredondamento para converter para 0. ou 1.
    return 'f1',f1_score(t,y_bin,average='weighted')

xgb_clf = xgb.XGBClassifier(max_depth=15, learning_rate=0.004,
                            n_estimators=200,
                            booster='gbtree',
                            silent=True,   objective='binary:logistic',
                            nthread=-1, gamma=0,
                            min_child_weight=1, max_delta_step=0, subsample=0.8,
                            colsample_bytree=0.6,
                            base_score=0.5,
                            seed=0, missing=None)

start = time.time()

xgb_clf.fit(X_train, y_train, eval_metric=xgb_f1,
         eval_set=[(X_train, y_train)],
         early_stopping_rounds=900)


end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_pred = xgb_clf.predict(X_test)


f1 = f1_score(y_pred, y_test, average='weighted')


print("F1 Score: " + str(round(f1,4)))

[0]	validation_0-error:0.227791	validation_0-f1:0.771055
Multiple eval metrics have been passed: 'validation_0-f1' will be used for early stopping.

Will train until validation_0-f1 hasn't improved in 900 rounds.
[1]	validation_0-error:0.191549	validation_0-f1:0.808109
[2]	validation_0-error:0.179341	validation_0-f1:0.819855
[3]	validation_0-error:0.173023	validation_0-f1:0.826331
[4]	validation_0-error:0.170066	validation_0-f1:0.829188
[5]	validation_0-error:0.166705	validation_0-f1:0.832621
[6]	validation_0-error:0.165946	validation_0-f1:0.833344
[7]	validation_0-error:0.160917	validation_0-f1:0.838464
[8]	validation_0-error:0.158516	validation_0-f1:0.840838
[9]	validation_0-error:0.157808	validation_0-f1:0.841545
[10]	validation_0-error:0.155913	validation_0-f1:0.843389
[11]	validation_0-error:0.154902	validation_0-f1:0.844428
[12]	validation_0-error:0.155028	validation_0-f1:0.844288
[13]	validation_0-error:0.155812	validation_0-f1:0.843431
[14]	validation_0-error:0.153259	validatio

[139]	validation_0-error:0.132054	validation_0-f1:0.867354
[140]	validation_0-error:0.131903	validation_0-f1:0.867509
[141]	validation_0-error:0.131726	validation_0-f1:0.867679
[142]	validation_0-error:0.131701	validation_0-f1:0.867705
[143]	validation_0-error:0.131524	validation_0-f1:0.867881
[144]	validation_0-error:0.131372	validation_0-f1:0.868043
[145]	validation_0-error:0.130993	validation_0-f1:0.868426
[146]	validation_0-error:0.130892	validation_0-f1:0.868526
[147]	validation_0-error:0.131018	validation_0-f1:0.868398
[148]	validation_0-error:0.130791	validation_0-f1:0.86863
[149]	validation_0-error:0.130917	validation_0-f1:0.86851
[150]	validation_0-error:0.130766	validation_0-f1:0.868659
[151]	validation_0-error:0.130715	validation_0-f1:0.868711
[152]	validation_0-error:0.130766	validation_0-f1:0.868656
[153]	validation_0-error:0.130437	validation_0-f1:0.868988
[154]	validation_0-error:0.130235	validation_0-f1:0.8692
[155]	validation_0-error:0.130007	validation_0-f1:0.869425
[

O algoritmo XGBClassifier não apresentou um bom desempenho. Apresentou o F1 Score final de apenas <b>71,55%</b>

### Deste modo, os próximos testes seguirão com os 3 melhores algoritmos considerando o F1 Score:
<b>
   - SVM Linear:    88,89%
   - SVM SVC:       87,37%
   - MultinomialNB: 86,17% 
</b>

# Próximo teste: Retirar stop words

A próxima estratégia será a retirada de stop words do português utilizando spacy e a execução dos 3 melhores algoritmos

In [15]:
# Tirando stop words utilizando o spacy 
# Gerando novamente os vetores de teste  

#Carrega base de português do spacy

pt = spacy.load('pt_core_news_sm')

nlp = spacy.load('pt')

#Referência as stop words do spacy 
stop_words_spacy = nlp.Defaults.stop_words

#Tokeniza com TF-IDF já excluindo as stop words 
vect_stop = TfidfVectorizer(ngram_range=(1,1), use_idf=True,stop_words=stop_words_spacy)
vect_stop.fit(df.text_pt)
text_vect_stop = vect_stop.transform(df.text_pt)

In [16]:
#Faz o novo split da nova amostra 
X_train_stop,X_test_stop,y_train_stop,y_test_stop = train_test_split(
    text_vect_stop, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [17]:
#Testa com SVM Linear mas com stop words já excluídas #

svm_linear_stop = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_stop.fit(X_train_stop, y_train_stop)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_stop.predict(X_test_stop)

f1 = f1_score(y_prediction, y_test_stop, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 4.045002460479736s
F1 Score: 0.8816


O teste sem stop words gerou um F1 Score ligeiramente menor <b>88,16%</b>

In [18]:
#Testa com SVM SVC mas com stop words já excluídas

# ATENÇÃO: Elevado tempo de execução 

svm_clf_stop = SVC(C=100, kernel='linear',random_state =42)

start = time.time()

svm_clf_stop.fit(X_train_stop, y_train_stop)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_clf_stop.predict(X_test_stop)

f1 = f1_score(y_prediction, y_test_stop, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 4086.915600538254s
F1 Score: 0.8647


Assim como no caso anterior, um resultado inferior ao apresentado com as stop words. F1 Score = <b>86,47%</b>

In [19]:
# Testa com multinomialNB mas com as stop words já excluídas

naive_multi_stop = MultinomialNB()

start = time.time()

naive_multi_stop.fit(X_train_stop,y_train_stop)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_stop.predict(X_test_stop)

f1 = f1_score(y_prediction, y_test_stop, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.10301756858825684s
F1 Score: 0.8611


O mesmo comportamento dos anteriores. Apresentou aqui um resultado inferior daquele com stop words. F1 Score = <b>86,11%</b>

<br>
<br>
## Desse modo, a estratégia de excluir as stop words não se mostrou efetiva

## A próxima estratégia será utilizar stemizadores e verificar os seus efeitos.

## Como o algoritmo SVM SVC tem desempenho consistentemente inferior ao LinearSVC e demora em torno de 2 horas para ser executado, ele será excluído da lista de melhores algoritmos. 

## Seguiremos com os algoritmos: LinearSVC e MultinomialNB
<br>
<br>


In [20]:
#Realiza o download dos stemizadores rslp e porter
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\cauee\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [21]:
#Converte cada frase no Dataframe para a sua versão Stemizada pelo RSLPStemmer
rslp = RSLPStemmer()

#Função que converte o texto original para o stematizador RSLP
def conv_stem(texto):
  return ' '.join([rslp.stem(token) for token in texto.split(' ')])

#Cria uma nova coluna no Dataframe com as frases stematizadas
df['stemizado'] = df.text_pt.apply(conv_stem)

In [22]:
#Tokeniza as frases stemizadas

#Vetoriza o texto utilizando TF-IDF em unigramas
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df.stemizado)
text_vect = vect.transform(df.stemizado)

In [23]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_stem,X_test_stem,y_train_stem,y_test_stem = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [24]:
#Testa com SVM Linear as frases lematizadas com RSLP

svm_linear_stem = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_stem.fit(X_train_stem, y_train_stem)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_stem.predict(X_test_stem)

f1 = f1_score(y_prediction, y_test_stem, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 7.192000150680542s
F1 Score: 0.8777


O resultado apresentado é inferior àquele com as stop words. O resultado não foi satisfatório. F1 Score atingido = <b>87,77%</b>

In [25]:
# Testa com multinomialNB
naive_multi_stem = MultinomialNB()

start = time.time()

naive_multi_stem.fit(X_train_stem,y_train_stem)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_stem.predict(X_test_stem)

f1 = f1_score(y_prediction, y_test_stem, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.10599970817565918s
F1 Score: 0.8546


O mesmo problema com o MultiNomialNB. Não foi satisfatório. F1 Score = <b>85,46%</b>

<br>
<br>
## Repete os mesmos testes para o stemizador Porter
<br>
<br>

In [26]:
#Cria mais uma coluna no Dataframe com as frases stematizadas pelo outro stemizador Porter

ps = PorterStemmer()

#Converte frases para o novo stemizador Porter
def conv_stem(texto):
  return ' '.join([ps.stem(token) for token in texto.split(' ')])

#Carrega na nova coluna criada stemizado2
df['stemizado2'] = df.text_pt.apply(conv_stem)

In [27]:
#Vetoriza essa nova stemização

#Vetoriza o texto utilizando TF-IDF em unigramas
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df.stemizado2)
text_vect = vect.transform(df.stemizado2)

In [28]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_stem2,X_test_stem2,y_train_stem2,y_test_stem2 = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [29]:
#Testa com SVM Linear

svm_linear_stem2 = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_stem2.fit(X_train_stem2, y_train_stem2)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_stem2.predict(X_test_stem2)

f1 = f1_score(y_prediction, y_test_stem2, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 8.279999017715454s
F1 Score: 0.885


A estratégia não foi bem sucedida. O F1 Score obtido não foi o suficiente. F1 Score obtido = <b>88,50%</b>

In [30]:
# Testa com multinomialNB

naive_multi_stem2 = MultinomialNB()

start = time.time()

naive_multi_stem2.fit(X_train_stem2,y_train_stem2)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_stem2.predict(X_test_stem2)

f1 = f1_score(y_prediction, y_test_stem2, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.11599946022033691s
F1 Score: 0.8583


O mesmo comportamento. F1 Score inferior. Valor obtido = <b>85,83%</b>

<br>
<br>

## Uma nova estratégia a ser testada é acrescentar informações sintáticas junto às palavras que compõem o texto. 
<br>
<br>


In [31]:
# Acrescentando informação da análise sintática 
# Gerando novamente os vetores de teste  

# ATENÇÃO: Elevado tempo de execução

#Converte frase para o seguinte padrão <palavra original>-<classe sintática> 
def conv_sintatico(texto):
    doc = pt(texto)
    str = ''
    for token in doc:
        str += token.text + '-' + token.pos_ + ' '
    return str 

start = time.time()

#Cria uma nova coluna chamada sintatico com esse padrão
df['sintatico'] = df.text_pt.apply(conv_sintatico)

end = time.time()
print("Tempo de adição do POS Tagger: " + str(end - start) + "s")

Tempo de adição do POS Tagger: 3101.849513530731s


In [32]:
#Vetoriza os textos com a análise sintática

#Vetoriza o texto utilizando TF-IDF em unigramas
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df.sintatico)
text_vect = vect.transform(df.sintatico)

In [33]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_sint,X_test_sint,y_train_sint,y_test_sint = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [34]:
#Testa com SVM Linear

svm_linear_sint = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_sint.fit(X_train_sint, y_train_sint)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_sint.predict(X_test_sint)

f1 = f1_score(y_prediction, y_test_sint, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 31.670559406280518s
F1 Score: 0.8885


O resultados não foram suficientes. Ligeiramente menores. F1 Score = <b>88,85%</b>

In [35]:
# Testa com multinomialNB

naive_multi_sint = MultinomialNB()

start = time.time()

naive_multi_sint.fit(X_train_sint,y_train_sint)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_sint.predict(X_test_sint)

f1 = f1_score(y_prediction, y_test_sint, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.11400175094604492s
F1 Score: 0.8624


In [36]:
#Atribui o dataset que será trabalhado 
df = df_original 

#converte todas as palavras para minúsculo porque não há diferença entre maiúsculas e minúsculas 
#para capturar o sentimento contido nas frases
df.text_pt = df.text_pt.str.lower()

<br>
<br>

## Outro teste realizado foi utilizar Word2Vec de 300 posições a partir de um base carregada da Internet ao invés do TF-IDF como método de vetorização
<br>
<br>


In [37]:
#Faz download da base a partir da Internet

tar_gz_path = '../extras/cbow_s300.zip'

class DLProgress(tqdm):
  last_block = 0

  def hook(self, block_num=1, block_size=1, total_size=None):
    self.total = total_size
    self.update((block_num - self.last_block) * block_size)
    self.last_block = block_num

if not isfile(tar_gz_path):
  with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Word2Vec Model') as pbar:
    urlretrieve(
      'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip',
      tar_gz_path,
      pbar.hook)

if not isfile('../extras/cbow_s300.txt'):     
  zip_ref = zipfile.ZipFile(tar_gz_path, 'r')
  zip_ref.extractall('../extras/')
  zip_ref.close()

In [38]:
#Tendo a base baixada, carrega para a memória

start = time.time()

model_cbow = KeyedVectors.load_word2vec_format('../extras/cbow_s300.txt')

end = time.time()
print("Tempo de carregamento: " + str(end - start) + "s")

Tempo de carregamento: 227.7136194705963s


In [39]:
#Converte uma frase em um único vetor
#Faz essa conversão pela soma dos vetores de cada palavras. 
#A soma será padronizada a partir da raiz quadrada da  média dos elementos ao quadrado

def conv_word2vec_frase(frase):
    soma =  np.zeros(model_cbow.vector_size)
    rms = 0
    
        
    for palavra in frase.split(' '):
        palavra = palavra.translate(palavra.maketrans('', '', string.punctuation))    
        try:
            soma = soma + model_cbow[palavra]    
        except:
            soma = soma + 0
        
    rms = 0
    
    for i in range (model_cbow.vector_size):
        rms = rms + (soma[i]*soma[i])
            
    rms = math.sqrt(rms / model_cbow.vector_size)
        
    word2vec_frase = soma / rms
        
            
    return word2vec_frase

In [40]:
#Cria uma nova coluna na Dataframe para aplicar a conversão de frase em um vetor através do word2vec
df['word2vec'] = df.text_pt.apply(conv_word2vec_frase)

In [41]:
#Converte o dataframe em uma matriz
n_linhas = df.word2vec.values.shape[0]
n_colunas = model_cbow.vector_size

word2vec_matriz = np.empty((n_linhas,n_colunas))
for i in range(n_linhas):
    elemen = df.word2vec[i]
    for j in range(n_colunas):
        word2vec_matriz[i][j] = elemen[j]

In [42]:
#Faz a divisão da matriz + classificação em amostra de treino e teste

#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_cbow,X_test_cbow,y_train_cbow,y_test_cbow = train_test_split(
    word2vec_matriz, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [43]:
#Testa com SVM Linear sobre o dados convertidos via word2vec

svm_linear_cbow = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_cbow.fit(X_train_cbow, y_train_cbow)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_cbow.predict(X_test_cbow)

f1 = f1_score(y_prediction, y_test_cbow, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 35.94609713554382s
F1 Score: 0.7961


Os resultados foram ruins. Foi obtido apenas <b>79,61%</b> de F1 Score

In [44]:
# Testa com multinomialNB
# Como o algoritmo não permite vetores com números negativos
# Realizo uma translação no vetor para ficar maior ou igual a zero

#Faz translação antes de executar
min_train = np.min(X_train_cbow) 
min_test = np.min(X_test_cbow)

X_train_cbow2 = X_train_cbow - min_train
X_test_cbow2 = X_test_cbow - min_test

naive_multi_cbow = MultinomialNB()

start = time.time()

naive_multi_cbow.fit(X_train_cbow2,y_train_cbow)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_cbow.predict(X_test_cbow2)

f1 = f1_score(y_prediction, y_test_cbow, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.10351777076721191s
F1 Score: 0.6736


O resultado com o MultiNomialNB é muito pior, ficando abaixo do mínimo solicitado. Apenas <b>67,36%</b> de F1 Score 

<br>
<br>
## A estratégia agora será rodar uma rede neural (LSTM) em Keras para verificar se obtem melhor performance que as obtidas até então
<br>
<br>

In [45]:
max_fatures = 1300
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df.text_pt.values)
X = tokenizer.texts_to_sequences(df.text_pt.values)
X = pad_sequences(X)

In [46]:
def f1_keras(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

                 
    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
                 
                 
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [47]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = [f1_keras])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 777, 128)          166400    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 777, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 421,594
Trainable params: 421,594
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
Y = pd.get_dummies(df.sentiment).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)

In [49]:
# ATENÇÃO: Elevado tempo de execução e elevado consumo de memória

batch_size = 512

start = time.time()

model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

Epoch 1/15
 - 177s - loss: 0.5509 - f1_keras: 0.7100
Epoch 2/15
 - 172s - loss: 0.3985 - f1_keras: 0.8275
Epoch 3/15
 - 167s - loss: 0.4220 - f1_keras: 0.8218
Epoch 4/15
 - 165s - loss: 0.3952 - f1_keras: 0.8300
Epoch 5/15
 - 166s - loss: 0.3591 - f1_keras: 0.8497
Epoch 6/15
 - 165s - loss: 0.3518 - f1_keras: 0.8518
Epoch 7/15
 - 165s - loss: 0.3474 - f1_keras: 0.8539
Epoch 8/15
 - 165s - loss: 0.3375 - f1_keras: 0.8596
Epoch 9/15
 - 166s - loss: 0.3276 - f1_keras: 0.8642
Epoch 10/15
 - 166s - loss: 0.3207 - f1_keras: 0.8692
Epoch 11/15
 - 166s - loss: 0.3026 - f1_keras: 0.8751
Epoch 12/15
 - 166s - loss: 0.3012 - f1_keras: 0.8765
Epoch 13/15
 - 166s - loss: 0.3087 - f1_keras: 0.8720
Epoch 14/15
 - 166s - loss: 0.3289 - f1_keras: 0.8615
Epoch 15/15
 - 165s - loss: 0.2895 - f1_keras: 0.8811
Tempo de treinamento: 2503.5361399650574s


In [50]:
_,score_f1 = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print("f1_score: %.4f" % (score_f1))

f1_score: 0.8635


O resultado não foi o suficiente para suplantar os resultados obtidos até então. O F1 Score foi de <b>86,35%</b>

<br>
<br>
## Mais uma estratégia, aumentar a quantidade de informações sobre os textos utilizando unigramas e bigramas nos modelos de classificação
## Esse enriquecimento será feito nos textos originais (com stop words) que foi o que se mostrou mais promissor até então.
<br>
<br>

In [51]:
#### Teste com unigramas e bigramas e os 2 melhores algoritmos ####

#Vetoriza o texto utilizando TFID em unigramas e digramas
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True)
vect.fit(df.text_pt)
text_vect = vect.transform(df.text_pt)

In [52]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_diag,X_test_diag,y_train_diag,y_test_diag = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [53]:
#Testa com SVM Linear
#### MELHOR COM 89,48% ####

svm_linear_diag = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_diag.fit(X_train_diag, y_train_diag)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_diag.predict(X_test_diag)

f1 = f1_score(y_prediction, y_test_diag, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 10.329999923706055s
F1 Score: 0.8949


Excelente resultado !! Melhor F1 Score obtido até então. <b>Novo valor máximo = 89,49%</b>

In [54]:
# Testa com multinomialNB

naive_multi_diag = MultinomialNB()

start = time.time()

naive_multi_diag.fit(X_train_diag,y_train_diag)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_diag.predict(X_test_diag)

f1 = f1_score(y_prediction, y_test_diag, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.38502931594848633s
F1 Score: 0.884


Muito bom resultado também. F1 Score = <b>88,40%</b>

<br>
## O teste agora será verificar se a inclusão de trigramas juntamente com unigramas e bigramas aumenta ainda mais o F1 Score
<br>

In [4]:
#### Teste com unigramas,bigramas e trigramas e os 2 melhores algoritmos ####

#Vetoriza o texto utilizando TFID em unigramas e digramas
vect = TfidfVectorizer(ngram_range=(1,3), use_idf=True)
vect.fit(df.text_pt)
text_vect = vect.transform(df.text_pt)

In [5]:
#Treina com a proporção de 80% para treinamento e 20% para teste
X_train_trigr,X_test_trigr,y_train_trigr,y_test_trigr = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 42
)

In [6]:
#Testa com SVM Linear


svm_linear_trigr = LinearSVC(penalty='l1',dual=False,C=1.0, random_state =42)

start = time.time()

svm_linear_trigr.fit(X_train_trigr, y_train_trigr)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_trigr.predict(X_test_trigr)

f1 = f1_score(y_prediction, y_test_trigr, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 16.038594007492065s
F1 Score: 0.8909


Com o SVM Linear ocorre uma piora ao acrescentar os trigramas. O F1 Score abaixou para <b>89,09%</b>

In [7]:
# Testa com multinomialNB

naive_multi_trigr = MultinomialNB()

start = time.time()

naive_multi_trigr.fit(X_train_trigr,y_train_trigr)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = naive_multi_trigr.predict(X_test_trigr)

f1 = f1_score(y_prediction, y_test_trigr, average='weighted')

print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 0.743952751159668s
F1 Score: 0.887


Com o MultiNomialNB o efeito foi ao contrário, há uma melhoria no F1 Score que alcança o valor de <b>88,70%</b>

<br>
## O uso de trigramas não alcançou resultados consistentes para que se prosseguisse com essa estratégia. 
## Agora serão utilizadas estratégias de melhora dos hiperparâmetros e tentativas de alteração no método de treinamento (K-Fold) com o algoritmo de LinearSVC e a vetorização baseada em unigramas e bigramas
<br>

In [55]:
#### Com otimização através de GridSearch no classificador LinearSVC ####
#### MELHOR 90,97% de f1 score #####

parametros = {'penalty': ['l1', 'l2'],
              'C': [1.0,2.0,4.0] }


svm_linear_opt = GridSearchCV(svm_linear_diag, parametros, scoring='f1_weighted')

start = time.time()

svm_linear_opt.fit(X_train_diag, y_train_diag)

end = time.time()
print("Tempo de treinamento: " + str(end - start) + "s")

y_prediction = svm_linear_opt.predict(X_test_diag)

f1 = f1_score(y_prediction, y_test_diag, average='weighted')
print("F1 Score: " + str(round(f1,4)))

Tempo de treinamento: 231.4320569038391s
F1 Score: 0.9097


Com o modelo otimizado pela otimização dos hiperparâmetros (GridSearchCV). <b>Obtemos o novo melhor máximo = 90,97%</b>

In [57]:
# ATENÇÃO: Elevado tempo de execução 

# Teste do melhor método com K-FOLD 
X_kfold = X_train_diag
#Y_kfold = y_train.as_matrix()
Y_kfold = y_train_diag.values
    
kf = StratifiedKFold(n_splits=40,random_state=42,shuffle=True)

clf = svm_linear_opt


best_model = None 
best_f1 = -1 

# start = time.time()

for train_index, test_index in kf.split(X_kfold,Y_kfold):  
    X_train_kfold, X_test_kfold = X_kfold[train_index], X_kfold[test_index]
    y_train_kfold, y_test_kfold = Y_kfold[train_index], Y_kfold[test_index]
    
    
    clf.fit(X_train_kfold, y_train_kfold)
    y_prediction = clf.predict(X_test_kfold)
    f1 = f1_score(y_prediction, y_test_kfold, average='weighted')
    
    if f1 > best_f1:
       best_f1 = f1
       best_model = deepcopy(clf)
    print("f1 obtido em treinamento...")    
    print(f1)

# end = time.time()
# print("Tempo de treinamento: " + str(end - start) + "s")

X_final_test = X_test_diag 
#Y_final_test = y_test_diag.as_matrix()
Y_final_test = y_test_diag.values


y_pred = best_model.predict(X_final_test)

print("F1 final com a amostra de teste....")
f1 = f1_score(y_pred,Y_final_test,average='weighted')

print("F1 Score: " + str(round(f1,4)))

f1 obtido em treinamento...
0.9050582560296846
f1 obtido em treinamento...
0.9121243504860276
f1 obtido em treinamento...
0.9040877191251182
f1 obtido em treinamento...
0.9000145928641914
f1 obtido em treinamento...
0.9181900836650582
f1 obtido em treinamento...
0.9111263506992067
f1 obtido em treinamento...
0.9181847401076808
f1 obtido em treinamento...
0.9120420703618893
f1 obtido em treinamento...
0.916082851939068
f1 obtido em treinamento...
0.9261962199588266
f1 obtido em treinamento...
0.8928208121917877
f1 obtido em treinamento...
0.8999078942935429
f1 obtido em treinamento...
0.90495974719088
f1 obtido em treinamento...
0.918149196895327
f1 obtido em treinamento...
0.9100414049792268
f1 obtido em treinamento...
0.8928269489599285
f1 obtido em treinamento...
0.912051067362934
f1 obtido em treinamento...
0.9059856237327915
f1 obtido em treinamento...
0.910029252819553
f1 obtido em treinamento...
0.8969140500331428
f1 obtido em treinamento...
0.919110046937198
f1 obtido em treinam

<br>
Mesmo utilizando treinamento com a estratégia de K-FOLD não se conseguiu superar o modelo otimizado com divisão simples de amostras de treinamento e validação. Nesse último teste, obteve-se F1 Score = <b>90,95%</b>. Quase o mesmo valor obtido anteriormente, mas ainda assim inferior. 

<br>
<br>
<br>
# Resultado Final

# Melhor algoritmo: SVM - LinearSVC

# Melhor estratégia: Vetorizar utilizando unigramas e bigramas e TF-IDF

# Otimização: Tuning dos hiperparâmetros do modelo LinearSVC utilizando GridSearchCV

# F1 Score final: 90,97%
<br>
<br>
<br>